# postcovid dataset preprocessing
### Import

## Train IMPACT model

In [1]:
from liriscat.utils import generate_eval_config
import json
from IMPACT.dataset import LoaderDataset as IMPACT_dataset
from liriscat.CDM import *
from IMPACT import model
from liriscat.dataset.preprocessing_utilities import *

CUDA is not available. Skipping CUDA seed setting.
importing json on engine(s)
importing utils,model,dataset from IMPACT on engine(s)
importing logging on engine(s)
importing gc on engine(s)
importing torch on engine(s)


In [2]:
folds_nb = 5
dataset_name="postcovid"

# Set all the required parameters ---------------
IMPACT_config = generate_eval_config(num_epochs=200, save_params=True, dataset_name=dataset_name, embs_path="../embs/"+dataset_name, params_path="../ckpt/"+dataset_name,  learning_rate=0.02026, lambda_=1.2e-5, batch_size=2048,valid_metric='rmse', pred_metrics=["rmse"])

concept_map, metadata, nb_modalities = load_dataset_resources(IMPACT_config)


# Conversion helper that builds a DataFrame with specific dtypes and returns records.
#def convert_to_records(data):
#    df = pd.DataFrame(data, columns={'user_id': int, 'item_id': int, 'dimension_id': int,'correct': float, })
#    return df.to_records(index=False, column_dtypes={'user_id': int, 'item_id': int, 'correct': float, 'dimension_id': int})


CUDA is not available. Using CPU.


In [3]:
dataset_name = 'postcovid'
for i in range(folds_nb):

    horizontal_train, horizontal_valid = horizontal_data(IMPACT_config, i)

    impact_train_data = IMPACT_dataset(horizontal_train, concept_map, metadata, nb_modalities)
    impact_valid_data = IMPACT_dataset(horizontal_valid, concept_map, metadata, nb_modalities) # <---

    IMPACT_config['i_fold'] = i
    algo = model.IMPACT(**IMPACT_config)
    algo.init_model(impact_train_data, impact_valid_data)
    algo.train(impact_train_data, impact_valid_data)
    print(algo.evaluate_predictions(impact_valid_data))

CUDA is not available. Skipping CUDA seed setting.


 18%|█▊        | 37/200 [00:24<01:45,  1.54it/s]


KeyboardInterrupt: 